### 计算PNL（一）、 计算手续费收入

1. 选定一个池子，获取池子的基本信息 ✅
2. 选定一个头寸和时间范围之后，获得这段时间内池子里的所有swap记录 ✅
3. 获取池子流动性的时间序列数据 ✅
4. 结合3和4计算出选定的头寸在这段时间内的手续费收入 

In [141]:

import sys 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.getcwd()))
# 这是为了自动重载自定义包的内容（python好不方便呀）
import importlib
import poolData.config
import poolData.swapData as swapData

importlib.reload(poolData.config)
importlib.reload(swapData)

<module 'poolData.swapData' from '/Users/bcfh/Documents/GitHub/learn_uniswapv3/poolData/swapData.py'>

In [142]:
from datetime import datetime
begin = datetime(2022, 12, 31)
end = datetime(2023, 1, 1)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"

### 1. 获取池子的基本信息

https://etherscan.io/address/0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#readContract

手续费率：0.05%

protocol fee: 0.05%

### 2. 获取所有swpa记录

In [115]:
data = swapData.query_swaps(int(begin.timestamp()), int(end.timestamp()), pool_id)
data

,id,amount0,amount1,timestamp,amountUSD,sqrtPriceX96
0,0x00001ac287e0a1b0a82d8b8445ad6493aa45a5a0f37f...,-220,0.165164902440265255,1673417903,219.9457656936104222193963066212759,2170293927366776459289426094439454
1,0x0000306bc3246038da552285fa7dc7c910eb286c2bbe...,-95,0.076043182366132083,1672817147,95.04077918628346236667773176495035,2240986979075360632018086744267947
2,0x0000cde32d93c27e0b8909faa3deff77b39c196552d3...,41168.561785,-32.082468149638960158,1673221199,41195.11344969503995011027414999015,2212240139497217658436759529264508
3,0x0001413726ac7ed8bd835c7b163fb52460ad00c56c33...,-31781.533974,23.897635823703337387,1673371691,31791.08979702467295888129380414709,2172015826983119174420027583272803
4,0x000191fbd21e8da2b6cb51bd625dd9ab462eb6e74e14...,-1717.630879,1.282788143576881559,1673377739,1717.915649927105113181969853897741,2164631118775481483705428312241414
...,...,...,...,...,...,...
95,0x003ef12c061d76124ef41d610a4f3776a0018b6fc89d...,2227.814903,-1.78,1672805879,2226.999923853596727015241936229526,2240051861601888178174697679757806
96,0x003fd7a928c75aa99b940a3e1ad58c485786ce88bfb7...,60,-0.05031823987435225,1672549151,60.0095160782350119430598651889871,2294960781592820084510836519724884
97,0x00400f376b71f5029dcaf50544cd24547ade502eb31b...,2000,-1.644969394353264072,1672748207,1999.857716493226106356122856375675,2272750701080177707836206335705243
98,0x0041193305bb9435cad66a58ac127754a93fd50678b3...,-7192.06716,6.0186195228691949,1672432187,7190.16302773826849461823163521779,2291361640263562465927969846766925


### 3. 获取池子流动性的时间序列数据

In [139]:
data = swapData.query_liquidity(int(begin.timestamp()), int(end.timestamp()), pool_id)
data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1672419600,43383970859571409238,2292741488478236193954710948079555,1194.122792146251869705949121256262,0.0008374348153950348453709271537854136,205468,2186941498930609995212528368989333,333609426.6969159360891818260545127,0,0,0,0,225,1194.511809050204970771435064936336,1195.135863873698740398661399390146,1193.804391237179988357396971990688,1194.122792146251869705949121256262
1,1672423200,41908297001142436164,2289914226033824321765933548671387,1197.073280553630480706658776824827,0.000835370746507275888776747615886998,205444,2186952021398667747348751967478818,333488757.0211926141408956767590597,0,0,0,0,260,1194.123444963926810270341103316625,1197.073280553630480706658776824827,1194.080786262999696020970218231062,1197.073280553630480706658776824827
2,1672426800,43259411757668858895,2292095502081045216146554614279866,1194.795971584433226686676411440843,0.0008369629826202778712332352962748905,205463,2186954913044984967466704176819519,333637095.322414727186961463831933,0,0,0,0,289,1197.073280553630480706658776824827,1197.07382117549381742907326713768,1194.750385533700741374110953471534,1194.795971584433226686676411440843
3,1672430400,41853986357057955972,2290601372199165019827373276558852,1196.355180105557782621625188956052,0.0008358721695941226979286513996652267,205450,2186962943960877559886609212538257,333906894.4645702736186860023230338,0,0,0,0,263,1194.796131311705043167380267771029,1196.355180105557782621625188956052,1194.693649299187396892654724716834,1196.355180105557782621625188956052
4,1672434000,59109260845469698637,2288829923181971195179725662338341,1198.207744422359329581249063675693,0.0008345798169432523948316314255772096,205434,2186981817612323087894790173617207,334440380.7305554361225720897040363,0,0,0,0,338,1196.394407853511805846627602601061,1200.916144596237448207735051238417,1196.394407853511805846627602601061,1198.207744422359329581249063675693
5,1672437600,42635286778298209243,2289840993430537869827068272854702,1197.149850232257000874432172955406,0.0008353173162123285613392883684579638,205443,2186985466783348684010015616485416,334159110.5215818807642015737513923,0,0,0,0,284,1198.206832687504706963694948656337,1198.788882736739192695900153120176,1196.572152892231302126497021777375,1197.149850232257000874432172955406
6,1672441200,42266969974656527701,2288242077353913887971525628274079,1198.823458909347496249755693340487,0.0008341511776135654455497975422011697,205429,2186997074931896828965175654912667,334188221.1952228731733906900809821,0,0,0,0,270,1197.82449644040087986756220591501,1199.244261599893039949720652423436,1197.824418695737796824551141160768,1198.823458909347496249755693340487
7,1672444800,42375952704822449015,2289495616475923548060168410771042,1197.511064425397845989152045989084,0.0008350653532206237699360214312313554,205440,2187003010215967611186322600569947,334113410.6901975287201155154576156,0,0,0,0,276,1198.825324382086128819092982674148,1198.842790767155624227608013196213,1197.056358947755097837177674114186,1197.511064425397845989152045989084
8,1672448400,41797181380235020195,2290877490356118393288567624905111,1196.066805470759543825298400099456,0.0008360737004204462694587878968945214,205452,2187005870139216598257742201908262,334115162.1019867095005736493665301,0,0,0,0,246,1197.510920373478583418711445267929,1198.119081909486525458850696326448,1196.065922163147825920797406752285,1196.066805470759543825298400099456
9,1672452000,41798684703065372303,2290870471734989472712595852616980,1196.074134346587037321554399359016,0.0008360685774266810361037704755810068,205452,2187009235706317547008239273028613,334125073.6342072299745118809878181,0,0,0,0,237,1195.859701519310864535825172283601,1196.098191300317605999126383310542,1195.154928442939157034554203522369,1196.074134346587037321554399359016


### 4. 计算头寸的手续费分成收入

参数解释：
1. 不要忘记protocol fee(需要从池子合约中获取)

    swap的手续费不是所有都给流动性提供者的，一部分会给协议，最终由dao收取。

2. feeGrowthGlobal0X128 和 feeGrowthGlobal1X128

    feeGrowthGlobal0X128:  The all-time global fee growth, per unit of liquidity, in token0

    feeGrowthGlobal1X128:  The all-time global fee growth, per unit of liquidity, in token1

    这是池子合约中的两个全局状态变量，池子每一次swap都会自动更新这两个值，表示的是整个池子里手续费的累计增长率，X128表示这个值是乘$ 2 ^ {128} $之后的值，所以在实际使用的时候需要把这个值除以$ 2 ^ {128} $.

    feeGrowthGlobal的计算方法：

    $$ feeGrowthGlobal0X128 = \frac{accumulative fees \times 2 ^ {128}}{total liquidity} $$

3. feeGrowthOutside0X128 和 feeGrowthOutside1X128

    这两个变量的定义和上面的global差不多，差别是feeGrowthOutside0X128是记录一个tick外所有的手续费增长率

    知道了feeGrowthOutside0X128和feeGrowthGlobal0X128的值，就可以计算出每次swap之后该position能获得的手续费是多少。
    
考虑两种情况：

a. swap是在你的tick range内发生的

$$ feetoken0 = \frac {(feeGrowthGlobal0X128 - feeGrowthOutside0X128_lower - feeGrowthOutside0X128_upper - feeGrowthInside0LastX128) \times liquidity}{ 2 ^ {128} \times 10 ^ {decimal}} $$

b. swap发生后你的tick out of range了

    i. current ticker > tick upper
        $$ token0fee = \frac {(feeGrowthOutside0X128_upper - feeGrowthOutside0X128_lower - feeGrowthInside0LastX128) \ times liquidity } { 2 ^ {128}  \times 10 ^ {decimal} } $$
    
    ii. current ticker < tick upper
        $$ token0fee = \frac {(feeGrowthOutside0X128_lower - feeGrowthOutside0X128_upper - feeGrowthInside0LastX128) \ times liquidity } { 2 ^ {128}  \times 10 ^ {decimal} } $$

***注意***

在the graph中能拿到的最细粒度数据是小时级别的，用以上的计算方法存在一个误差点：

利用小时级别的feeGrowth数据来计算手续费收入，忽视了一个小时内池子流动性发生的变化。如果在这一个小时内，有流动性提供者推出或者新的流动性提供者加入，那么feeGrowth数据其实是会发生变化的。

但这个误差是可以容忍的，所以我们假设一个小时内池子的流动性不发生变化。

4. fee

在v2中，收取的手续费会自动转变成liquidity，随着池子内swap不断发生，你的liquidity也会逐渐积累。

但在v3中，收取的手续费不再自动转成liquidity，背后的原因是：virtual reserve的存在导致收取的手续费无法直接按比例加入池子成为流动性。

fee的思考：

v3的特点就是允许你在某几个tick内提供流动性，需要一个比较好的方式来存储以tick为单位的手续费，所以v3记录手续费的方式是用全局变量记录整个池子内的手续费增长情况 + 每个tick的手续费增长情况，和直接记录手续费大小相比这是一个优化后的数据结构设计。这一部分的内容在白皮书的6.3左右：

swap的流程：
![swap-control](formulas/swap-control.png)

对于每一个tick，合约中都会存储以下这些变量：

1. liquidityNet：

    the total amount of liquidity that should be kicked in or out when the tick is crossed. 
    $$ \Delta L = \frac {} $$
2. liquidityGross
3. feeGrowthOutside0X128
4. feeGrowthOutside1X128
5. secondsOutside
6. tickCumulativeOutside
7. secondsPerLiquidityOutsideX128



### 5. 获取tick的数据

参数解释：

1. tick id：

$$ i $$



In [ ]:
data = swapData.query_ticks(pool_id)

## 关于回测框架

计算头寸的收益最难的就是手续费的部分了，但是如果有position feeGrowthOutside1X128的时间序列数据，结合池子的feeGrowthGlobal0X128对时间序列数据，可以直接计算出一个头寸的手续费收入。

不过为了分析不同策略的pnl（如果不自己写一个的话，只能用现实中真实发生过的头寸进行分析，没有办法做一些策略的设计），我们需要一个回测框架，支持模拟各种头寸的设置，假设头寸不影响池子的深度和价格的情况下分析头寸的pnl。

那么如何计算手续费呢？

1. 获取池子内所有的swap数据，筛选出在你的头寸之内的所有swap
2. 计算该swap会给你的头寸带来多少手续费收入

**但是我突然有个大胆的想法🙋🏽**

uniswap中的池子不同于传统的金融市场，传统金融市场中做回测是很简单的。比如你想做一个股票的回测框架，你可以不用考虑你的交易对市场价格产生的影响，因为上市的股票往往有着足够的深度和流动性。但是在uniswap中，许多池子都非常非常小，你的头寸很可能会大幅度影响币对的价格。

如果只研究池子深度足够的池子里什么策略好，那上面所说的回测框架是足够了的，但是研究出来的结果对于深度小的池子肯定是不适用的。

能不能自己部署一个uniswap合约，交易数据可以用uniswap上的实际交易数据或者用中心化交易所的交易数据，把这些数据放到自己部署的合约上重放。

